# FINAL CRAWLER (crawling TOROB.COM)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

number = int(input("please enter number of items that you need in search result-> "))

print("please wait to open torob.com...!")

options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome(options=options)
driver.get("https://torob.com")

#کوئری زدن به وبسایت ترب با استفاده از بسته سلنیوم
query = input("Please enter your query to torob.com -> ")
search_element=driver.find_element(By.CSS_SELECTOR,"#search-query-input")
search_element.send_keys(query)
search_element.send_keys(Keys.ENTER)

print("Sending query to torob's server")

time.sleep(5)

#گرفتن تعداد آیتم های موجود در صفحه نتایج
n = len(driver.find_elements(By.CSS_SELECTOR,"#layout-wrapp > div.jsx-3806219686.content.false > div > div > div.jsx-2899210753.left-container > div.jsx-2899210753.cards-wrapper > div > div > div.jsx-2834913897"))
print(n)

#بررسی برای این که آیا به اسکرول نیاز داریم یا نه ؟
if number > 24:
    #scroll down
    my_element = driver.find_element(By.TAG_NAME,"body")
    
    while True:
        my_element.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        n = len(driver.find_elements(By.CSS_SELECTOR,"#layout-wrapp > div.jsx-3806219686.content.false > div > div > div.jsx-2899210753.left-container > div.jsx-2899210753.cards-wrapper > div > div > div.jsx-2834913897"))
        if(n>=number):
            print(n)
            break
#######################################################################################اتمام اسکرول و آغاز استخراج نتایج

soup = BeautifulSoup(driver.page_source, 'html.parser')


print("crawling items page...")

#link
temp = soup.select("#layout-wrapp > div.jsx-3806219686.content.false > div > div > div.jsx-2899210753.left-container > div.jsx-2899210753.cards-wrapper > div > div > div.jsx-2834913897 > a")[:number]
link = ["https://torob.com/"+i['href'] for i in temp]

#title
temp = soup.select("#layout-wrapp > div.jsx-3806219686.content.false > div > div > div.jsx-2899210753.left-container > div.jsx-2899210753.cards-wrapper > div > div > div.jsx-2834913897 > a > div > div > h2")[:number]
title = [i.text for i in temp]

#price
temp = soup.select("#layout-wrapp > div.jsx-3806219686.content.false > div > div > div.jsx-2899210753.left-container > div.jsx-2899210753.cards-wrapper > div > div > div.jsx-2834913897 > a > div > div > div.jsx-2834913897.table-cell > div.jsx-2834913897")[:number]
price = [i.text for i in temp]

#Stores_count
temp = soup.select("#layout-wrapp > div.jsx-3806219686.content.false > div > div > div.jsx-2899210753.left-container > div.jsx-2899210753.cards-wrapper > div > div > div.jsx-2834913897 > a > div.jsx-2834913897 > div.jsx-2834913897 > div.jsx-2834913897.table-cell > div.jsx-2834913897.shops")[:number]

Stores_count = [i.text for i in temp]

print(link)
print(title)
print(price)
print(Stores_count)
###############################################################################################اتمام استخراج نتایج صفحه نتایج 
###############################################################################################باز کردن لینک هر نتیجه و استخراج اطلاعات آن

print("crawling each items page...")
item_desc, item_image = [],[]

for link1 in tqdm(link):
    try:
        site = requests.get(link1)
    except:
        time.sleep(2)
        site = requests.get(link1)
        
    soup = BeautifulSoup(site.text, "html.parser")


    #item_image
    temp = soup.select("#layout-wrapp > div.jsx-3806219686.content.false > div > div.jsx-785696898.jsx-21171105.grid-container-with-seller > div.jsx-785696898.jsx-21171105.showcase > div > div > div.jsx-222558948.gallery > div > div > span > img")
    item_image.append(temp[0]['src'])
        
    #item_desc
    temp = soup.select("#layout-wrapp > div.jsx-3806219686.content.false > div > div.jsx-785696898.jsx-21171105.grid-container-with-seller > div.jsx-785696898.jsx-21171105.showcase > div > div > div.jsx-222558948.main-info > div.jsx-222558948.name > h1")
    item_desc.append(temp[0].text)

#######################################################################################اتمام استخراج اطلاعات هر نتیجه و آغاز بارگیری
print("downloading author images...")
items = list(set(zip(item_desc, item_image)))
i=1
for item in tqdm(items):
    try:
        file = requests.get(item[1])
    except:
        time.sleep(5)
        file = requests.get(item[1])
    with open("C:\\Crawling\\Presentation2\\Files\\Images\\"+str(i)+".jpg", 'wb') as f:
        f.write(file.content)
    i+=1
#######################################################################################


#######################################################################################ایجاد فرم مناسب برای ذخیره اطلاعات
print("building DataFrame...")
data = pd.DataFrame({
    'title':title,
    'store_count':Stores_count,
    'price':price,


    'item_desc':item_desc,
    'item_image':item_image,

    
    'link':link
})
data.to_csv("C:\\Crawling\\Presentation2\\Files\\Images\\data.csv", index=None, encoding='utf-8')
print("finished!")

please enter number of items that you need in search result-> 10
please wait to open torob.com...!
Please enter your query to torob.com -> یخچال
Sending query to torob's server
24
crawling items page...
['https://torob.com//p/47579213-b86b-481d-b9c9-8088151f247c/%DB%8C%D8%AE%DA%86%D8%A7%D9%84-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1-%D8%AF%D9%88%D9%88-%D9%85%D8%AF%D9%84-d4bf-1077/', 'https://torob.com//p/8dc20801-53f8-457c-8687-8f45b4b2437c/%DB%8C%D8%AE%DA%86%D8%A7%D9%84-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1-%D8%A8%D8%A7%D9%84%D8%A7-%D8%B3%DB%8C%D9%86%D8%AC%D8%B1/', 'https://torob.com//p/a3b253e7-7c31-4f3d-919f-514f88b9855e/%DB%8C%D8%AE%DA%86%D8%A7%D9%84-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1-%D8%A7%DB%8C%D8%B3%D8%AA%DA%A9%D9%88%D9%84-%D9%85%D8%AF%D9%84-tm-679-200/', 'https://torob.com//p/522e7344-bc6c-4c67-8f58-b6e3d19d8998/%DB%8C%D8%AE%DA%86%D8%A7%D9%84-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1-%D9%87%DB%8C%D9%85%D8%A7%D9%84%DB%8C%D8%A7-%D9%85%D8%AF%D9%84-%DA%A9%D9%85%D8%A8%DB%8C-530-%D9%

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.25it/s]


downloading author images...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.57it/s]

building DataFrame...
finished!
